# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster',con=engine)
X = df.message
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize text
    tokens = word_tokenize(text) 
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens
    pass



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier())),
        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [6]:
model.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:

y_test_pred = model.predict(X_test)

In [8]:
print(classification_report(y_test,y_test_pred,target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4930
               request       0.77      0.43      0.56      1094
                 offer       0.00      0.00      0.00        29
           aid_related       0.74      0.58      0.65      2682
          medical_help       0.59      0.09      0.15       531
      medical_products       0.86      0.08      0.14       317
     search_and_rescue       0.53      0.04      0.08       185
              security       0.50      0.01      0.02       102
              military       0.78      0.12      0.21       233
                 water       0.81      0.39      0.52       429
                  food       0.83      0.45      0.58       714
               shelter       0.79      0.24      0.37       569
              clothing       0.88      0.07      0.13        98
                 money       0.67      0.03      0.06       133
        missing_people       0.60      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
accuracy = (y_test == y_test_pred).mean().mean()
print("Accuracy:", accuracy)

Accuracy: 0.9432479308


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
def model_pipeline2():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    
    parameters = {
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [20,30]
        }
    cv = GridSearchCV(pipeline, param_grid=parameters)
    return cv

In [11]:
model2 = model_pipeline2()
model2.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:

y_test_pred = model2.predict(X_test)
def display_results(cv, y_test, y_pred):
    accuracy = (y_pred == y_test).mean().mean()

    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)
    print(classification_report(y_test,y_pred,target_names=y.columns))


In [13]:
display_results(model2,y_test,y_test_pred)

Accuracy: 0.945943928517

Best Parameters: {'clf__estimator__n_estimators': 30, 'features__text_pipeline__tfidf__use_idf': True}
                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      4930
               request       0.82      0.49      0.61      1094
                 offer       0.00      0.00      0.00        29
           aid_related       0.74      0.67      0.70      2682
          medical_help       0.63      0.08      0.15       531
      medical_products       0.69      0.07      0.13       317
     search_and_rescue       0.78      0.04      0.07       185
              security       0.00      0.00      0.00       102
              military       0.82      0.10      0.18       233
                 water       0.90      0.33      0.49       429
                  food       0.82      0.53      0.64       714
               shelter       0.80      0.33      0.46       569
              clothing       0.80     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
def model_pipeline3():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(GradientBoostingClassifier())),
        ])
    return pipeline

In [15]:
model3 = model_pipeline3()
model3.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
           n_jobs=1))])

In [16]:
y_test_pred = model3.predict(X_test)
print(classification_report(y_test,y_test_pred,target_names=y.columns))
accuracy = (y_test == y_test_pred).mean().mean()
print("Accuracy:", accuracy)

                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      4930
               request       0.80      0.50      0.61      1094
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.55      0.64      2682
          medical_help       0.62      0.20      0.30       531
      medical_products       0.67      0.26      0.37       317
     search_and_rescue       0.29      0.16      0.21       185
              security       0.10      0.07      0.08       102
              military       0.71      0.27      0.39       233
                 water       0.71      0.65      0.68       429
                  food       0.77      0.76      0.76       714
               shelter       0.78      0.52      0.62       569
              clothing       0.63      0.57      0.60        98
                 money       0.39      0.21      0.27       133
        missing_people       0.33      

### 9. Export your model as a pickle file

In [17]:
# Dump the trained decision tree classifier with Pickle
randomforest_pkl_filename = 'randomforest_classifier_20190118.pkl'
# Open the file to save as pkl file
randomforest_pkl_model_pkl = open(randomforest_pkl_filename, 'wb')
pickle.dump(model2, randomforest_pkl_model_pkl)
# Close the pickle instances
randomforest_pkl_model_pkl.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.